<a href="https://colab.research.google.com/github/thefr33radical/hybrid_forecasting/blob/master/code/ARIMA_ANN_lynx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install statsmodels
import pandas as pd 
import numpy as np
from google.colab import files
import io
from statsmodels.tsa.seasonal import seasonal_decompose
from matplotlib import pyplot as plt
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score,mean_squared_error as ms
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima_model import ARIMA

data = files.upload()
data = pd.read_csv(io.BytesIO(data["sunspot.csv"]),index_col=0)
print(data.columns)
data=data.set_index("time")
print(data)


In [0]:
data.plot()
plt.show()

In [0]:
linear_data = ndata.trend+ ndata.seasonal 
linear_data.plot()
plt.show()

In [0]:
# ARIMA individual

train_data =  non_linear_data.iloc[2:221,:]
test_data =  non_linear_data.iloc[272:-2,:]

model_arima = ARIMA(train_data, order=(9,0,0))
model_fit = model_arima.fit(disp=0)
residuals = pd.DataFrame(model_fit.resid)
residuals.plot()
residuals.plot(kind='kde')
plt.show()
print(residuals.describe())
model_fit.summary()

#y_train_pred = model_fit.forecast(x_train)
y_test_pred = model_fit.forecast()
print(y_test_pred)

In [0]:
non_linear_data = ndata.resid 
non_linear_data.plot()
plt.show()

In [0]:
# ANN individual

train_data =  non_linear_data.iloc[2:221,:]
test_data =  non_linear_data.iloc[222:-2,:]

#train_data =  data.iloc[2:221,:]
#test_data =  data.iloc[222:-2,:]

#scaler = MinMaxScaler(feature_range=(-1, 1))
#train_data = scaler.fit_transform(train_data)
#test_data = scaler.fit_transform(test_data)

x_train = train_data[:-1]
y_train = train_data[1:]

x_test = test_data[:-1]
y_test = test_data[1:]

model = Sequential()
model.add(Dense(7,input_dim=1,activation="sigmoid"))
model.add(Dense(5))
model.add(Dense(1))

model.compile(loss="mean_squared_error",optimizer="Adam")
early_stop = EarlyStopping(monitor ="loss",patience =10, verbose =1)
model.fit(x_train,y_train, epochs =2, batch_size=15,verbose=1,shuffle= False)
y_test_pred = model.predict(x_test)

y_train_pred = model.predict(x_train)
print("The MSE score on the Train set is:\t{:0.3f}".format(ms(y_train, y_train_pred)))
print("The MSE score on the Test set is:\t{:0.3f}".format(ms(y_test, y_test_pred)))

In [0]:
# ARIMA Hybrid

train_data =  non_linear_data.iloc[2:221,:]
test_data =  non_linear_data.iloc[272:-2,:]

model_arima = ARIMA(train_data, order=(9,0,0))
model_fit = model_arima.fit(disp=0)
residuals = pd.DataFrame(model_fit.resid)
residuals.plot()
residuals.plot(kind='kde')
plt.show()

print(residuals.describe())
model_fit.summary()
y_test_pred = model_fit.forecast()
print(y_test_pred)

In [0]:
# ANN Hybrid
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score,mean_squared_error as ms
from sklearn.preprocessing import MinMaxScaler

train_data =  non_linear_data.iloc[2:221,:]
test_data =  non_linear_data.iloc[222:-2,:]

x_train = train_data[:-1]
y_train = train_data[1:]

x_test = test_data[:-1]
y_test = test_data[1:]

model = Sequential()
model.add(Dense(7,input_dim=1,activation="sigmoid"))
model.add(Dense(5))
model.add(Dense(1))

model.compile(loss="mean_squared_error",optimizer="Adam")
early_stop = EarlyStopping(monitor ="loss",patience =10, verbose =1)
model.fit(x_train,y_train, epochs =2, batch_size=15,verbose=1,shuffle= False)

y_test_pred = model.predict(x_test)
y_train_pred = model.predict(x_train)

print("The MSE score on the Train set is:\t{:0.3f}".format(ms(y_train, y_train_pred)))
print("The MSE score on the Test set is:\t{:0.3f}".format(ms(y_test, y_test_pred)))